In [8]:
#!pip install mysql
#!pip install mysql-connector-python-rf
#import warnings
#warnings.filterwarnings("ignore")

You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [155]:
from flask import Flask
from flask import url_for, render_template, request, redirect
import mysql.connector as mysql



In [156]:
session = {}
session['username'] = None

In [167]:
app = Flask(__name__)

In [168]:
con = mysql.connect(user = 'root', password = 'password', 
                    host = '127.0.0.1', database = 'db_proj', #database = 'task_manager'
                   auth_plugin='mysql_native_password')
cur = con.cursor(dictionary=True)

In [169]:
@app.route('/home', methods=['GET', 'POST'])
def index():
    if session['username'] == None:
        return render_template('home.html')
    else:
        tasks = []
        importance = []
        statuses = []
        ids = []
        base_string = "SELECT id, task, importance, status FROM (SELECT user_tasks.id_user_task as \'id\', tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + session['username'] + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance) as t"
        if request.method == "POST":
            det = request.form
##            with open ('test.txt', 'a') as f:
##                f.write(det['importance1'])
            importance_list = []
            status_list = []
            if 'importance1' in det:
                importance_list.append(det['importance1'])
            if 'importance2' in det:
                importance_list.append(det['importance2'])
            if 'importance3' in det:
                importance_list.append(det['importance3'])
    
            if 'status1' in det:
                status_list.append(det['status1'])
            if 'status2' in det:
                status_list.append(det['status2'])
            if 'status3' in det:
                status_list.append(det['status3'])
           
            if not importance_list and not status_list:
                 
                cur.execute(base_string)
                result = cur.fetchall()
            else:
                base_string += ' WHERE ('
                if importance_list:
                    for i, element in enumerate(importance_list):
                        base_string = base_string + 'importance = \'' + element + '\''
                        if i != len(importance_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if status_list:
                    if importance_list:
                        base_string += ' AND ('
                    for i, element in enumerate(status_list):
                        base_string = base_string + 'status = \'' + element + '\''
                        if i != len(status_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
            cur.execute(base_string)  
            result = cur.fetchall()
        else:
            cur.execute(base_string)
            result = cur.fetchall()
        data = []
        for res in result:
            elem = [res['task'], res['importance'], res['status'], res['id']]
            data.append(elem)
#            ids.append(res['id'])
#            tasks.append(res['task'])
#            importance.append(res['importance'])
#            statuses.append(res['status'])
        return render_template('home_logged_in.html', data = data)

In [170]:
@app.route('/friends_tasks', methods=['GET', 'POST'])
def my_friends_tasks():
    tasks = []
    importance = []
    statuses = []
    ids = []
    
    # еще джойн (получаем список юзернеймов друзей юзера):
    cur.execute("SELECT username from users_list \
                JOIN (select id2 from users_list, user_relations \
                where id1 = id_user and username = '" + session['username'] + "') as f_id \
                ON id2 = id_user")  
    res = cur.fetchall()
    friends_list = [result['username'] for result in res]
    
    if friends_list == []: # если нет никаких друзей
        return render_template('no_friends.html')
    
    else: # если есть хотя бы один друг
        base_string = "SELECT username, task, importance, status FROM \
        (SELECT username, tasks_list.task_name as 'task', \
        importance_list.importance_name as 'importance', status_list.status_name as 'status' \
        FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
        WHERE ("
        for id_f, f_username in enumerate(friends_list):
            base_string = base_string + "users_list.username = '" + f_username + "'"
            if id_f != len(friends_list) - 1:
                base_string += ' OR '
            else:
                base_string += ')'
        end_of_subquery = " and users_list.id_user = user_tasks.id_user \
        and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
        and importance_list.id_importance = user_tasks.id_importance) as t"
        base_string += end_of_subquery
        
        if request.method == "POST":
            det = request.form
            importance_list = []
            status_list = []
            friend = det['friend']
        
            if 'importance1' in det:
                importance_list.append(det['importance1'])
            if 'importance2' in det:
                importance_list.append(det['importance2'])
            if 'importance3' in det:
                importance_list.append(det['importance3'])
    
            if 'status1' in det:
                status_list.append(det['status1'])
            if 'status2' in det:
                status_list.append(det['status2'])
            if 'status3' in det:
                status_list.append(det['status3'])
           
            if not importance_list and not status_list and friend == 'all':            
                cur.execute(base_string)
                result = cur.fetchall()
            else:
                base_string += ' WHERE ('
                if importance_list:
                    for i, element in enumerate(importance_list):
                        base_string = base_string + "importance = '" + element + "'"
                        if i != len(importance_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if status_list:
                    if importance_list:
                        base_string += ' AND ('
                    for i, element in enumerate(status_list):
                        base_string = base_string + "status = '" + element + "'"
                        if i != len(status_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if friend != 'all':
                    if importance_list or status_list:
                        base_string = base_string + " AND (username = '" + friend + "')"
                    else:
                        base_string = base_string + "username = '" + friend + "')"
                        
            cur.execute(base_string)  
            result = cur.fetchall()
        else:
            cur.execute(base_string)
            result = cur.fetchall()
        data = []
        for res in result:
            elem = [res['task'], res['importance'], res['status'], res['username']]
            data.append(elem)

        return render_template('friends_tasks.html', data = data, friends = friends_list)

In [171]:
@app.route('/signup', methods=['GET', 'POST'])
def sign_up():
    if request.method == "GET":
        details = request.form
        username = details['username']
        password = details['password']
        cur.execute("INSERT INTO users_list (username, password) VALUES (%s, %s)", (username, password))
        con.commit()
        cur.close()
        return redirect('login')
    return render_template('sign_up.html')

In [172]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":
        details = request.form
        username = details['username']
        password = details['password']
        cur.execute("SELECT * FROM users_list")
        users = cur.fetchall()
        user_exists = 0
        for i in users:
            if i['username'] == username:
                user_exists = 1
                if i['password'] == password:
                    session['username'] = username
                    return redirect('home')
                else:
                    return ('Неправильный пароль.')
        if user_exists == 1:
            return redirect ('home')
        else:
            return ('Такого пользователя пока нет.')
    return render_template('login.html')

In [173]:
@app.route('/new_task', methods=['GET', 'POST'])
def new_task(): 
    if request.method == "POST":
        details = request.form
        cur.execute("SELECT id_user FROM users_list WHERE username = '" + session['username'] + "'")
        res = cur.fetchone()
        id_user = res['id_user']

##        cur.execute("SELECT id_task FROM tasks_list WHERE task_name = '" + details['tasks'] + "'")
##        res = cur.fetchone()
##       id_task = res['id_task']
        
        cur.execute("SELECT id_status FROM status_list WHERE status_name = '" + details['status'] + "'")
        res = cur.fetchone()
        id_status = res['id_status']
        
        cur.execute("SELECT id_importance FROM importance_list WHERE importance_name = '" + details['importance'] + "'")
        res = cur.fetchone()
        id_importance = res['id_importance']
        
        cur.execute("SELECT task_name, id_task FROM tasks_list")
        res = cur.fetchall()
        task_exists = 0
        for i in res:
            if i['task_name'] == details['tasks']:
                task_exists = 1
                id_task = i['id_task']
        if task_exists == 0:        
            cur.execute("INSERT INTO tasks_list (task_name) VALUES ('" + details['tasks'] + "')")
            cur.execute("SELECT id_task FROM tasks_list WHERE task_name = '" + details['tasks'] + "'")
            res = cur.fetchone()
            id_task = res['id_task']
        data = (id_user, id_task, id_status, id_importance)
        cur.execute("INSERT INTO user_tasks (id_user, id_task, id_status, id_importance) VALUES (%s, %s, %s, %s)", data)
        con.commit()
        return redirect('home')
    importance = []
    status = []
    cur.execute("SELECT importance_name FROM importance_list")
    res = cur.fetchall()
    for i in res:
        importance.append(i['importance_name'])
    cur.execute("SELECT status_name FROM status_list")
    res = cur.fetchall()
    for i in res:
        status.append(i['status_name'])
    return render_template('new_task.html', status = status, importance = importance)

In [174]:
@app.route('/update', methods=['GET', 'POST'])
def update_task():
    user_task_id = request.args.get('id') # должно быть в этом месте
    if request.method == "GET":
        # ура, джойн!:
        cur.execute("select task_name from tasks_list join user_tasks on \
        user_tasks.id_task = tasks_list.id_task and id_user_task = " + user_task_id)
        res = cur.fetchone()
        task = res['task_name']
    if request.method == "POST":
        details = request.form
        
        cur.execute("SELECT id_status FROM status_list WHERE status_name = '" + details['status'] + "'")
        res = cur.fetchone()
        id_status = res['id_status']
        
        cur.execute("SELECT id_importance FROM importance_list WHERE importance_name = '" + details['importance'] + "'")
        res = cur.fetchone()
        id_importance = res['id_importance']
        
        cur.execute("UPDATE user_tasks SET id_status = " + str(id_status) + ", id_importance = " + str(id_importance) + " \
        WHERE id_user_task = " + str(user_task_id))
        con.commit()
        return redirect('home')
    
    importance = []
    status = []
    cur.execute("SELECT importance_name FROM importance_list")
    res = cur.fetchall()
    for i in res:
        importance.append(i['importance_name'])
    cur.execute("SELECT status_name FROM status_list")
    res = cur.fetchall()
    for i in res:
        status.append(i['status_name'])
    return render_template('update_task.html', status = status, importance = importance, task = task)

In [175]:
@app.route('/delete', methods=['POST', 'GET'])
def delete():
    if request.args.get('id'):
        task_id = request.args.get('id')
        cur.execute("DELETE FROM user_tasks WHERE id_user_task = '" + task_id + "'")
        con.commit()
        return redirect('home')
    else:
        return('this page is meant to delete something')        

In [176]:
if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Dec/2019 04:56:01] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:56:11] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [26/Dec/2019 04:56:11] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:56:35] "GET /friends_tasks HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:56:46] "POST /friends_tasks HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:57:01] "POST /friends_tasks HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:57:05] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:57:16] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [26/Dec/2019 04:57:16] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:57:18] "GET /friends_tasks HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:57:23] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:57:31] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [26/Dec/2019 04:57:31] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 04:57:33] "GET /friends_tasks HTTP

Ниже фигня

In [131]:
cur.execute("select username from users_list \
            join (select id2 from users_list, user_relations \
            where id1 = id_user and username = 'mcqueen') as f_id \
            on id2 = id_user")  
res = cur.fetchall()
friends_list = [result['username'] for result in res]
friends_list

['fishowl', 'mashasuvorova']

In [133]:
base_string = "SELECT username, task, importance, status FROM \
(SELECT username, tasks_list.task_name as 'task', \
importance_list.importance_name as 'importance', status_list.status_name as 'status' \
FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE ("
for id_f, friend in enumerate(friends_list):
    
    base_string = base_string + "users_list.username = '" + friend + "'"
    if id_f != len(friends_list) - 1:
        base_string += ' OR '
    else:
        base_string += ')'
        
if True == True:
    base_string_end = " and users_list.id_user = user_tasks.id_user and \
    tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status and \
    importance_list.id_importance = user_tasks.id_importance) as t"
#print(base_string)
    base_string += base_string_end

    importance_list = ['важно', 'не очень важно']
    status_list = ['закончено']
    friend = None

    if not importance_list and not status_list and not friend:            
        cur.execute(base_string)
        result = cur.fetchall()
    else:
        base_string += ' WHERE ('
        if importance_list:
            for i, element in enumerate(importance_list):
                base_string = base_string + "importance = '" + element + "'"
                if i != len(importance_list) - 1:
                    base_string += ' OR '
                else:
                    base_string += ')'
        if status_list:
            if importance_list:
                base_string += ' AND ('
            for i, element in enumerate(status_list):
                base_string = base_string + "status = '" + element + "'"
                if i != len(status_list) - 1:
                    base_string += ' OR '
                else:
                    base_string += ')'
        if friend:
            if importance_list or status_list:
                base_string = base_string + " AND (username = '" + friend + "')"
            else:
                base_string = base_string + "username = '" + friend + "')"

print(base_string)
cur.execute(base_string)  
res = cur.fetchall()
res

SELECT username, task, importance, status FROM (SELECT username, tasks_list.task_name as 'task', importance_list.importance_name as 'importance', status_list.status_name as 'status' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list WHERE (users_list.username = 'fishowl' OR users_list.username = 'mashasuvorova') and users_list.id_user = user_tasks.id_user and     tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status and     importance_list.id_importance = user_tasks.id_importance) as t WHERE (importance = 'важно' OR importance = 'не очень важно') AND (status = 'закончено')


[{'importance': 'не очень важно',
  'status': 'закончено',
  'task': 'вырезать снежинку',
  'username': 'fishowl'}]

In [32]:
cur.execute("SELECT username, task, importance, status FROM \
(SELECT username, tasks_list.task_name as 'task', \
importance_list.importance_name as 'importance', \
status_list.status_name as 'status' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE (users_list.username = 'mashasuvorova' or users_list.username = 'fishowl') \
and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance) as t \
WHERE importance = 'не очень важно'") #\
#and username = 'mashasuvorova'")
res = cur.fetchall()
print(res)

[{'username': 'fishowl', 'task': 'вырезать снежинку', 'importance': 'не очень важно', 'status': 'закончено'}]


In [20]:
cur.execute("insert into users_list(id_user, username, password) values (4, 'ejouny', 'нееммясо')")
con.commit()

In [21]:
cur.execute('SELECT * FROM db_proj.users_list')
res = cur.fetchall()
print(res)

[{'id_user': 1, 'username': 'fishowl', 'password': 'йцукен'}, {'id_user': 2, 'username': 'mcqueen', 'password': 'фывапро'}, {'id_user': 3, 'username': 'mashasuvorova', 'password': 'колбаса'}, {'id_user': 4, 'username': 'ejouny', 'password': 'нееммясо'}]


In [30]:
con.commit()

In [24]:
cur.execute("SELECT * FROM db_proj.user_relations")
res = cur.fetchall()
print(res)

[{'id_user_relation': 1, 'id1': 1, 'id2': 2}, {'id_user_relation': 2, 'id1': 2, 'id2': 1}, {'id_user_relation': 3, 'id1': 2, 'id2': 3}, {'id_user_relation': 4, 'id1': 3, 'id2': 2}]


In [7]:
id_status = 3
id_importance = 3
user_task_id = 1
query = "UPDATE user_tasks SET id_status = " + str(id_status) + ", id_importance = " + str(id_importance) + " WHERE id_user_task = " + str(user_task_id)
print(query)
cur.execute(query)
print(query)

UPDATE user_tasks SET id_status = 3, id_importance = 3 WHERE id_user_task = 1
UPDATE user_tasks SET id_status = 3, id_importance = 3 WHERE id_user_task = 1


In [393]:
tasks = []
importance = []
statuses = []

cur.execute("SELECT user_tasks.id_user_task, tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + session['username'] + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance")
result = cur.fetchall()
result

[{'id_user_task': 1,
  'importance': 'очень важно',
  'status': 'в процессе',
  'task': 'вырезать снежинку'},
 {'id_user_task': 3,
  'importance': 'очень важно',
  'status': 'не начато',
  'task': 'сделать домашку'},
 {'id_user_task': 14,
  'importance': 'не очень важно',
  'status': 'не начато',
  'task': 'сделать домашку'},
 {'id_user_task': 16,
  'importance': 'важно',
  'status': 'не начато',
  'task': 'записаться к стоматологу'},
 {'id_user_task': 17,
  'importance': 'очень важно',
  'status': 'в процессе',
  'task': 'приготовить ужин'},
 {'id_user_task': 18,
  'importance': 'не очень важно',
  'status': 'в процессе',
  'task': 'приготовить ужин'},
 {'id_user_task': 19,
  'importance': 'не очень важно',
  'status': 'в процессе',
  'task': 'приготовить ужин'},
 {'id_user_task': 20,
  'importance': 'очень важно',
  'status': 'в процессе',
  'task': 'перебрать рис'},
 {'id_user_task': 21,
  'importance': 'не очень важно',
  'status': 'не начато',
  'task': 'помыть полы'},
 {'id_user_

In [231]:
cur.execute("SELECT tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + 'fishowl' + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance")
result = cur.fetchall()
result

[{'importance': 'очень важно',
  'status': 'не начато',
  'task': 'сделать домашку'},
 {'importance': 'не очень важно',
  'status': 'не начато',
  'task': 'сделать домашку'}]

In [101]:
importance_list = ['важно', 'очень важно']
status_list = ['в процессе']
base_string = "SELECT task, importance, status FROM (SELECT tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + session['username'] + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance) as t"

if not importance_list and not status_list:
    cur.execute(base_string)
else:
    base_string += ' WHERE ('
    if importance_list:
        for i, element in enumerate(importance_list):
            base_string = base_string + 'importance = \'' + element + '\''
            if i != len(importance_list) - 1:
                base_string += ' OR '
            else:
                base_string += ')'
    if status_list:
        if importance_list:
            base_string += ' AND ('
        for i, element in enumerate(status_list):
            base_string = base_string + 'status = \'' + element + '\''
            if i != len(status_list) - 1:
                base_string += ' OR '
            else:
                base_string += ')'
        
cur.execute(base_string)  
result = cur.fetchall()

print(base_string)

SELECT task, importance, status FROM (SELECT tasks_list.task_name as 'task', importance_list.importance_name as 'importance',status_list.status_name as 'status' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list WHERE users_list.username = 'fishowl' and users_list.id_user = user_tasks.id_user and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status and importance_list.id_importance = user_tasks.id_importance) as t WHERE (importance = 'важно' OR importance = 'очень важно') AND (status = 'в процессе')
